<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Student-Model" data-toc-modified-id="Student-Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Student Model</a></span><ul class="toc-item"><li><span><a href="#Data-processing" data-toc-modified-id="Data-processing-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data processing</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#Teacher-like-Model" data-toc-modified-id="Teacher-like-Model-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Teacher-like Model</a></span></li><li><span><a href="#Downsized-Model" data-toc-modified-id="Downsized-Model-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Downsized Model</a></span></li><li><span><a href="#Pruned-Model" data-toc-modified-id="Pruned-Model-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Pruned Model</a></span></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#ROC-AUC" data-toc-modified-id="ROC-AUC-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>ROC AUC</a></span></li><li><span><a href="#Compression-rate" data-toc-modified-id="Compression-rate-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Compression rate</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Results</a></span></li></ul></li></ul></li></ul></div>

# Completed in Google Colab

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
!pip install -q deepctr

In [3]:
!rm -R 'model-optimization'
!git clone https://github.com/tensorflow/model-optimization.git
!rm -R 'tensorflow_model_optimization'
!mv 'model-optimization/tensorflow_model_optimization' 'tensorflow_model_optimization'

Cloning into 'model-optimization'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 3393 (delta 17), reused 27 (delta 8), pack-reused 3344
Receiving objects: 100% (3393/3393), 1.02 MiB | 9.87 MiB/s, done.
Resolving deltas: 100% (2318/2318), done.


In [4]:
# The prune registry needs to be updated to be DeepCTR-aware. The following changes are made to:

# /content/tensorflow_model_optimization/python/core/sparsity/keras/prune_registry.py
#
# import deepctr.layers.utils
# import deepctr.layers.interaction
# import deepctr.layers.core
# 
# class PruneRegistry(object):
#   _LAYERS_WEIGHTS_MAP = {
#       deepctr.layers.utils.Hash: [],
#       deepctr.layers.utils.NoMask: [],
#       deepctr.layers.utils.Linear: ['kernel'],
#       deepctr.layers.utils.Add: [],
#       deepctr.layers.interaction.CrossNet: ['kernels'],
#       deepctr.layers.core.DNN: ['kernels'],
#       deepctr.layers.core.PredictionLayer: [],
#       ...
#   }

# /content/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py
#
# class PruneLowMagnitude(Wrapper):
#   def build(self, input_shape):
#     self.prunable_weights = []
#     raw_prunable_weights = self.layer.get_prunable_weights()
#     for raw_weight in raw_prunable_weights:
#       if isinstance(raw_weight, list):
#         self.prunable_weights.extend(raw_weight)
#       else:
#         self.prunable_weights.append(raw_weight)

input("Confirm the aforementioned changes are made: ")

Confirm the aforementioned changes are made: YES


'YES'

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!rm -R 'data'
!mkdir -p 'data/criteo'
!cp '/content/drive/My Drive/criteo/soft_targets_full.csv' 'data/criteo/'
!cp '/content/drive/My Drive/criteo/train.csv.zip' 'data/criteo/'
!unzip -d 'data/criteo/' 'data/criteo/train.csv.zip'

Archive:  data/criteo/train.csv.zip
  inflating: data/criteo/train.csv   


# Student Model


Нужно обучть небольшую модель на [soft таргетах](https://drive.google.com/file/d/1tBbPOUT-Ow9f3zTDApykGXYwt-KslYle/view?usp=sharing)  модели учителя, которая не сильно уступала бы в качестве учителю.

In [0]:
import os
import pandas as pd
import numpy as np
import zipfile
from collections import defaultdict

import tensorflow as tf

from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from deepctr.inputs import SparseFeat, DenseFeat
from deepctr.models import DCN

In [0]:
DATA_DIR_PATH = 'data/criteo'

TRAIN_DATA_PATH = os.path.join(DATA_DIR_PATH, 'train.csv')
SOFT_TARGETS_PATH = os.path.join(DATA_DIR_PATH, 'soft_targets_full.csv')

## Data processing

Данные на Train/Validation/Test нужно разбить как 80/10/10

In [9]:
dense_features_indices = [i for i in range(1, 14)]
sparse_features_indices = [i for i in range(14, 40)]

dense_features = ['c{}'.format(i) for i in dense_features_indices]
sparse_features = ['c{}'.format(i) for i in sparse_features_indices]
feature_names = dense_features + sparse_features

len(dense_features_indices), len(sparse_features_indices), len(feature_names)

(13, 26, 39)

In [0]:
sparse_features_dims = dict([
    ('c14', 1445),
    ('c15', 556),
    ('c16', 1130758),
    ('c17', 360209),
    ('c18', 304),
    ('c19', 21),
    ('c20', 11845),
    ('c21', 631),
    ('c22', 3),
    ('c23', 49223),
    ('c24', 5194),
    ('c25', 985420),
    ('c26', 3157),
    ('c27', 26),
    ('c28', 11588),
    ('c29', 715441),
    ('c30', 10),
    ('c31', 4681),
    ('c32', 2029),
    ('c33', 4),
    ('c34', 870796),
    ('c35', 17),
    ('c36', 15),
    ('c37', 87605),
    ('c38', 84),
    ('c39', 58187)])

assert sparse_features == list(sorted(sparse_features_dims.keys()))

In [11]:
data = pd.read_csv(TRAIN_DATA_PATH)
data.rename(columns=dict([(col, col[1:] if col[0] == '_' else col) for col in data.columns]), inplace=True)
print(len(data))

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )

mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

3664931


In [12]:
soft_targets_df = pd.read_csv(SOFT_TARGETS_PATH)
print(len(soft_targets_df))
assert soft_targets_df["id"].tolist() == data["id"].tolist()

3664931


In [0]:
data = pd.merge(data, soft_targets_df, on="id")

In [14]:
train, test = train_test_split(data, test_size=0.2, shuffle=False)
validation, test = train_test_split(test, test_size=0.5, shuffle=False)

print(len(train), len(validation), len(test))

2931944 366493 366494


In [0]:
def gen_model_input(df):
    return {
        name: 
        (pd.core.series.Series(df[name]) if name in sparse_features else np.array(df[name]))
        for name in feature_names}


def gen_model_hard_soft_target(df):
    HARD_TARGET_COL = 'c0'
    SOFT_TARGET_COL = 'prob'
    hard_target = pd.core.series.Series(df[HARD_TARGET_COL])
    soft_target = pd.core.series.Series(df[SOFT_TARGET_COL])
    return hard_target, soft_target

In [0]:
train_input = gen_model_input(train)
train_hard_target, train_soft_target = gen_model_hard_soft_target(train)

In [0]:
validation_input = gen_model_input(validation)
validation_hard_target, validation_soft_target = gen_model_hard_soft_target(validation)

In [0]:
test_input = gen_model_input(test)
test_hard_target, test_soft_target = gen_model_hard_soft_target(test)

## Model

Можно также использовать Pruning и/или Quantinization.

In [0]:
epochs = 1
batch_size = 1024

In [0]:
def eval_model_on(model, split):
    if split == "train":
        input_df = train_input
        hard_target_df = train_hard_target
        soft_target_df = train_soft_target
    elif split == "validation":
        input_df = validation_input
        hard_target_df = validation_hard_target
        soft_target_df = validation_soft_target
    elif split == "test":
        input_df = test_input
        hard_target_df = test_hard_target
        soft_target_df = test_soft_target
    else:
        raise ValueError(f"Unknown split: {split}.")
    
    predictions = model.predict(input_df, batch_size=batch_size)
    split_roc_auc_score = roc_auc_score(hard_target_df, predictions)
    split_mse_on_soft = mean_squared_error(soft_target_df, predictions)

    print(f"{split} roc_auc_score {split_roc_auc_score:.4f}")
    print(f"{split} mse_on_soft {split_mse_on_soft:.4f}")

    return {
        "roc_auc_score": split_roc_auc_score,
        "mse_on_soft": split_mse_on_soft,
    }

In [0]:
def save_and_compress_checkpoint(model, handle):
    raw_checkpoint_file = f"{handle}.h5"
    tf.keras.models.save_model(model, raw_checkpoint_file, include_optimizer=False)
    zip_checkpoint_file = f"{handle}.zip"
    with zipfile.ZipFile(zip_checkpoint_file, "w", compression=zipfile.ZIP_DEFLATED) as fzip:
        fzip.write(raw_checkpoint_file)

    raw_checkpoint_size = os.path.getsize(raw_checkpoint_file) / float(2**20)
    zip_checkpoint_size = os.path.getsize(zip_checkpoint_file) / float(2**20)
    print(f"raw_checkpoint_size: {raw_checkpoint_size:.2f} MB")
    print(f"zip_checkpoint_size: {zip_checkpoint_size:.2f} MB")

    return {
        "raw_checkpoint_size": raw_checkpoint_size,
        "zip_checkpoint_size": zip_checkpoint_size,
    }

In [0]:
def create_dcn_model_with_columns(feature_columns):
    model = DCN(feature_columns, feature_columns, cross_num=2,
                dnn_hidden_units=(128, 128), l2_reg_linear=0, l2_reg_embedding=0,
                l2_reg_cross=0, l2_reg_dnn=0, init_std=0.0001, seed=1024, 
                dnn_use_bn=True, dnn_activation="relu",
                task="binary")  # Although task is "regression" task="binary" only maps the outputs to (0, 1):
                                # https://github.com/shenweichen/DeepCTR/blob/master/deepctr/layers/core.py#L238

    model.compile(optimizer="adam",
                  loss="mean_squared_error",
                  metrics=["mse"])
    
    return model

In [0]:
def train_model(model, callbacks=[]):
    model.fit(train_input, train_soft_target,
              validation_data=(validation_input, validation_soft_target),
              batch_size=batch_size, epochs=epochs,
              use_multiprocessing=True,
              verbose=1,
              callbacks=callbacks)

### Teacher-like Model

In [24]:
high_embedd_feature_columns = [DenseFeat(feat, 1) for feat in dense_features] + \
                              [SparseFeat(feat, 
                                          vocabulary_size=min(vocab_size, 50000), 
                                          embedding_dim=min(int(6 * (vocab_size) ** (0.25)), 100), 
                                          use_hash=True, dtype='string') 
                               for feat, vocab_size in sparse_features_dims.items()]

teacher_model = create_dcn_model_with_columns(high_embedd_feature_columns)
teacher_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
c14 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
c15 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
c16 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
c17 (InputLayer)                [(None, 1)]          0                                            
______________________________________________________________________________________________

In [25]:
train_model(teacher_model)

2864/2864 [==============================] - 581s 203ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0022 - val_mse: 0.0022


In [26]:
teacher_model_validation_eval = eval_model_on(teacher_model, "validation")
teacher_model_test_eval = eval_model_on(teacher_model, "test")

validation roc_auc_score 0.7919
validation mse_on_soft 0.0022
test roc_auc_score 0.7898
test mse_on_soft 0.0022


In [27]:
teacher_model_sizes = save_and_compress_checkpoint(teacher_model, "teacher_model")

raw_checkpoint_size: 160.23 MB
zip_checkpoint_size: 149.81 MB


### Downsized Model

In [28]:
low_embedd_feature_columns = [DenseFeat(feat, 1) for feat in dense_features] + \
                             [SparseFeat(feat,
                                         vocabulary_size=min(vocab_size, 10000),
                                         embedding_dim=32,
                                         use_hash=True, dtype="string")
                              for feat, vocab_size in sparse_features_dims.items()]

student_model = create_dcn_model_with_columns(low_embedd_feature_columns)
student_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
c14 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
c15 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
c16 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
c17 (InputLayer)                [(None, 1)]          0                                            
____________________________________________________________________________________________

In [29]:
train_model(student_model)

2864/2864 [==============================] - 102s 36ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.0029 - val_mse: 0.0029


In [30]:
student_model_validation_eval = eval_model_on(student_model, "validation")
student_model_test_eval = eval_model_on(student_model, "test")

validation roc_auc_score 0.7889
validation mse_on_soft 0.0029
test roc_auc_score 0.7864
test mse_on_soft 0.0030


In [31]:
student_model_sizes = save_and_compress_checkpoint(student_model, "student_model")

raw_checkpoint_size: 15.68 MB
zip_checkpoint_size: 14.42 MB


### Pruned Model

In [0]:
import tensorflow_model_optimization
from tensorflow_model_optimization.python.core.api.sparsity import keras as sparsity

In [0]:
low_embedd_feature_columns = [DenseFeat(feat, 1) for feat in dense_features] + \
                             [SparseFeat(feat,
                                         vocabulary_size=min(vocab_size, 10000),
                                         embedding_dim=32,
                                         use_hash=True, dtype="string")
                              for feat, vocab_size in sparse_features_dims.items()]

pruned_model = create_dcn_model_with_columns(low_embedd_feature_columns)

In [34]:
N = 10
for n in range(1, N+1):
    iter_target_sparsity = 1.0 - 1.0 / n
    print(f"Iter {n}/{N} target_sparsity={iter_target_sparsity}")

    pruning_params = {
        "pruning_schedule": sparsity.ConstantSparsity(target_sparsity=iter_target_sparsity,
                                                      begin_step=0,
                                                      end_step=-1,
                                                      frequency=100)
    }

    pruned_model = sparsity.prune_low_magnitude(pruned_model, **pruning_params)
    pruned_model.compile(optimizer="adam",
                        loss="mean_squared_error",
                        metrics=["mse"])

    callbacks = [
        sparsity.UpdatePruningStep()
    ]
    train_model(pruned_model, callbacks)

    pruned_model = sparsity.strip_pruning(pruned_model)

Iter 1/10 target_sparsity=0.0
Instructions for updating:
Please use `layer.add_weight` method instead.
2864/2864 [==============================] - 294s 103ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.0029 - val_mse: 0.0029
Iter 2/10 target_sparsity=0.5
2864/2864 [==============================] - 292s 102ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.0023 - val_mse: 0.0023
Iter 3/10 target_sparsity=0.6666666666666667
2864/2864 [==============================] - 293s 102ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.0023 - val_mse: 0.0023
Iter 4/10 target_sparsity=0.75
2864/2864 [==============================] - 293s 102ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0022 - val_mse: 0.0022
Iter 5/10 target_sparsity=0.8
2864/2864 [==============================] - 292s 102ms/step - loss: 0.0010 - mse: 0.0010 - val_loss: 0.0021 - val_mse: 0.0021
Iter 6/10 target_sparsity=0.8333333333333334
2864/2864 [==============================] - 290s 101ms/step - loss: 0.0010 - mse

In [35]:
pruned_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
c14 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
c15 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
c16 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
c17 (InputLayer)                [(None, 1)]          0                                            
____________________________________________________________________________________________

In [36]:
pruned_model_validation_eval = eval_model_on(pruned_model, "validation")
pruned_model_test_eval = eval_model_on(pruned_model, "test")

validation roc_auc_score 0.7913
validation mse_on_soft 0.0023
test roc_auc_score 0.7890
test mse_on_soft 0.0025


In [37]:
pruned_model_sizes = save_and_compress_checkpoint(pruned_model, "pruned_model")

raw_checkpoint_size: 15.68 MB
zip_checkpoint_size: 3.01 MB


## Evaluation

Наша основная задача получить модель, которая 
* в терминах ROC AUC не намного хуже модели учителя, и в то же время 
* сильно меньше по размеру

### ROC AUC

Сравним ROC AUC модели ученика с показателем для учителя.

ROC AUC учителя: 0.802

In [0]:
TEACHER_ROC_AUC_SCORE = 0.802

quality_restoration_rate = pruned_model_test_eval["roc_auc_score"] / TEACHER_ROC_AUC_SCORE

### Compression rate

Пусть 
* $a$ - \# of the parameters in the original model $M$
* $a^{*}$ - \# of the parameters in compressed model $M^{*}$

тогда compression rate is $$\alpha(M,M^{*}) = \frac{a}{a^{*}}$$

Можно также посчитать comression rate просто как отношение фактических размеров моделей.

Размер модели учителя - 168MB


In [39]:
print(teacher_model_sizes)

{'raw_checkpoint_size': 160.23017120361328, 'zip_checkpoint_size': 149.80591011047363}


Так как в качестве teacher_model использовалась точно такая же модель, как и модель учителя (только обученая на 1% данных), и размеры raw checkpoint-ов приблизительно равны 168MB vs 160 MB, в качестве baseline-а оценки сжатия возьмем teacher_model_sizes["zip_checkpoint_size"].

In [0]:
TEACHER_ZIP_CHECKPOINT_SIZE = teacher_model_sizes["zip_checkpoint_size"]

compression_rate = TEACHER_ZIP_CHECKPOINT_SIZE / pruned_model_sizes["zip_checkpoint_size"]

### Results

### Prune with PolynomialDecay

#### Pruning only Embedding layers

final_sparsity=0.90
then
quality restoration rate=0.9815
compression rate=44.6119

if
final_sparsity=0.95
then
quality restoration rate=0.9793
compression rate=59.7270

if
final_sparsity=0.99
then
quality restoration rate=0.9701
compression rate=87.2720

#### Pruning Embedding + DNN layers

if 
final_sparsity=0.90
then
quality restoration rate=0.9813
compression rate=49.8008

#### Pruning Embedding + DNN + CrossNet layers

if 
final_sparsity=0.90
then
quality restoration rate=0.9809
compression rate=49.8549

#### Pruning Embedding + DNN + CrossNet + Linear layers

if 
final_sparsity=0.90
then
quality restoration rate=0.9817
compression rate=49.8821


### Iteratrive ConstantSparsity pruning

if
final_sparsity=0.90
then
quality restoration rate=0.9837
compression rate=49.7863

In [41]:
print(f"quality restoration rate={quality_restoration_rate:.4f}")
print(f"compression rate={compression_rate:.4f}")

quality restoration rate=0.9837
compression rate=49.7863
